In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import time

In [ ]:
train_data = pd.read_csv("datasets/titanic/train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("datasets/titanic/test.csv")
test_data.head()

In [ ]:
train = train_data["Age"]
# train = train.dropna()
train.shape

In [ ]:
train = train_data[["Pclass", "Sex", "Age", "Parch", "SibSp", "Survived"]]
train = train.dropna(how="any", axis=0)
# test = test_data[["Pclass", "Sex", "Age", "Parch", "SibSp"]]
# test = test.dropna(how="any", axis=0)

y = train["Survived"]

features = ["Pclass", "Sex", "Age", "Parch", "SibSp"]
X = pd.get_dummies(train[features])
X_test = pd.get_dummies(test_data[features])

y = tf.constant(np.array([y]).T)
X = tf.constant(np.array(X))

X_test = tf.constant(np.array(X_test))

In [ ]:
X = tf.keras.utils.normalize(X)
X_test = tf.keras.utils.normalize(X_test)

y.shape

In [73]:
def build_model(layers, nodes):
    NAME = f"{nodes}-nodes-{layers}-layers-{int(time.time())}"
    print(NAME)
    tensorboard = TensorBoard(log_dir=f"logs/{NAME}")
    model = tf.keras.models.Sequential()
    
    for l in range(layers):
        model.add(tf.keras.layers.Dense(nodes, activation="relu"))

    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.binary_crossentropy, metrics=["accuracy"])

    model.fit(X, y, epochs=100, batch_size=64, validation_split=0.2, verbose=1, callbacks=[tensorboard])

    return model

In [ ]:
nodes = [4, 8, 12, 16, 32]

for layer in range(15):
    for node in nodes:
        model = build_model(layer, node)

In [78]:
model = build_model(5, 32)

32-nodes-5-layers-1634215582
Epoch 1/100


2021-10-14 18:16:22.451357: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-14 18:16:22.451418: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-14 18:16:22.452361: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


3/9 [=========>....................] - ETA: 0s - loss: 0.6894 - accuracy: 0.5938

2021-10-14 18:16:22.976317: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-14 18:16:22.976360: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-14 18:16:23.021214: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-10-14 18:16:23.022313: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-10-14 18:16:23.023956: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/32-nodes-5-layers-1634215582/train/plugins/profile/2021_10_14_18_16_23

2021-10-14 18:16:23.024646: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/32-nodes-5-layers-1634215582/train/plugins/profile/2021_10_14_18_16_23/root.trace.json.gz
2021-10-14 18:16:23.026005: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/32-nodes-5-layers-1634

9/9 [==============================] - 1s 62ms/step - loss: 0.6858 - accuracy: 0.5902 - val_loss: 0.6762 - val_accuracy: 0.6084
Epoch 2/100
9/9 [==============================] - 0s 5ms/step - loss: 0.6768 - accuracy: 0.5902 - val_loss: 0.6668 - val_accuracy: 0.6084
Epoch 3/100
9/9 [==============================] - 0s 5ms/step - loss: 0.6753 - accuracy: 0.5902 - val_loss: 0.6622 - val_accuracy: 0.6084
Epoch 4/100
9/9 [==============================] - 0s 5ms/step - loss: 0.6721 - accuracy: 0.5902 - val_loss: 0.6598 - val_accuracy: 0.6084
Epoch 5/100
9/9 [==============================] - 0s 5ms/step - loss: 0.6701 - accuracy: 0.5902 - val_loss: 0.6580 - val_accuracy: 0.6084
Epoch 6/100
9/9 [==============================] - 0s 5ms/step - loss: 0.6677 - accuracy: 0.5919 - val_loss: 0.6533 - val_accuracy: 0.6154
Epoch 7/100
9/9 [==============================] - 0s 5ms/step - loss: 0.6652 - accuracy: 0.5954 - val_loss: 0.6474 - val_accuracy: 0.6294
Epoch 8/100
9/9 [=====================

In [79]:
predictions = model.predict([X_test]).T

for i in range(predictions.shape[1]):    
        if predictions[0,i] > 0.5:
            predictions[0,i] = 1
        else:
            predictions[0,i] = 0
       
predictions = predictions.astype(np.int)

Consider rewriting this model with the Functional API.


In [80]:
predictions = np.squeeze(predictions)


# output = pd.DataFrame({'Sex': np.squeeze(X_val.T[:, 3]), 'Predictions': predictions, 'Actual': np.squeeze(y_val)})
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('titanic.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
